In [0]:
#EMP DataFrame
empData = [(1,"Smith",10), (2,"Rose",20),
    (3,"Williams",10), (4,"Jones",30), (7,"yynes",70)
  ]
empColumns = ["emp_id","name","emp_dept_id"]
empDF = spark.createDataFrame(empData,empColumns)
empDF.show()

+------+--------+-----------+
emp_id| name|emp_dept_id|
+------+--------+-----------+
 1| Smith| 10|
 2| Rose| 20|
 3|Williams| 10|
 4| Jones| 30|
 7| yynes| 70|
+------+--------+-----------+

In [0]:
#DEPT DataFrame
deptData = [("Finance",10), ("Marketing",20),
    ("Sales",30),("IT",40)
  ]
deptColumns = ["dept_name","emp_dept_id"]
deptDF=spark.createDataFrame(deptData,deptColumns)  
deptDF.show()

+---------+-----------+
dept_name|emp_dept_id|
+---------+-----------+
 Finance| 10|
Marketing| 20|
 Sales| 30|
 IT| 40|
+---------+-----------+

In [0]:
empDF.join(deptDF, how = 'left', on='emp_dept_id' ).show()

+-----------+------+--------+---------+
emp_dept_id|emp_id| name|dept_name|
+-----------+------+--------+---------+
 10| 1| Smith| Finance|
 10| 3|Williams| Finance|
 20| 2| Rose|Marketing|
 30| 4| Jones| Sales|
 70| 7| yynes| null|
+-----------+------+--------+---------+

In [0]:

#Address DataFrame
addData=[(1,"1523 Main St","SFO","CA"),
    (2,"3453 Orange St","SFO","NY"),
    (3,"34 Warner St","Jersey","NJ"),
    (4,"221 Cavalier St","Newark","DE"),
    (5,"789 Walnut St","Sandiago","CA")
  ]
addColumns = ["emp_id","addline1","city","state"]
addDF = spark.createDataFrame(addData,addColumns)
addDF.show()

+------+---------------+--------+-----+
emp_id| addline1| city|state|
+------+---------------+--------+-----+
 1| 1523 Main St| SFO| CA|
 2| 3453 Orange St| SFO| NY|
 3| 34 Warner St| Jersey| NJ|
 4|221 Cavalier St| Newark| DE|
 5| 789 Walnut St|Sandiago| CA|
+------+---------------+--------+-----+

In [0]:
#SQL
empDF.createOrReplaceTempView("EMP")
deptDF.createOrReplaceTempView("DEPT")
addDF.createOrReplaceTempView("ADD")

spark.sql("select * from EMP e, DEPT d, ADD a " + \
    "where e.emp_dept_id == d.emp_dept_id and e.emp_id == a.emp_id") \
    .show()

+------+--------+-----------+---------+-----------+------+---------------+------+-----+
emp_id| name|emp_dept_id|dept_name|emp_dept_id|emp_id| addline1| city|state|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+
 1| Smith| 10| Finance| 10| 1| 1523 Main St| SFO| CA|
 2| Rose| 20|Marketing| 20| 2| 3453 Orange St| SFO| NY|
 3|Williams| 10| Finance| 10| 3| 34 Warner St|Jersey| NJ|
 4| Jones| 30| Sales| 30| 4|221 Cavalier St|Newark| DE|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+

In [0]:
empDF

Out[35]: DataFrame[emp_id: bigint, name: string, emp_dept_id: bigint]

In [0]:
addDF

Out[36]: DataFrame[emp_id: bigint, addline1: string, city: string, state: string]

In [0]:
#Join two DataFrames
empDF.join(addDF, on=['emp_id'], how='inner').show()

+------+--------+-----------+---------------+------+-----+
emp_id| name|emp_dept_id| addline1| city|state|
+------+--------+-----------+---------------+------+-----+
 1| Smith| 10| 1523 Main St| SFO| CA|
 2| Rose| 20| 3453 Orange St| SFO| NY|
 3|Williams| 10| 34 Warner St|Jersey| NJ|
 4| Jones| 30|221 Cavalier St|Newark| DE|
+------+--------+-----------+---------------+------+-----+

In [0]:
empDF.join(deptDF).where(empDF["emp_dept_id"] == deptDF["emp_dept_id"]) \
    .join(addDF).where(empDF["emp_id"] == addDF["emp_id"]) \
    .show()

+------+--------+-----------+---------+-----------+------+---------------+------+-----+
emp_id| name|emp_dept_id|dept_name|emp_dept_id|emp_id| addline1| city|state|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+
 1| Smith| 10| Finance| 10| 1| 1523 Main St| SFO| CA|
 2| Rose| 20|Marketing| 20| 2| 3453 Orange St| SFO| NY|
 3|Williams| 10| Finance| 10| 3| 34 Warner St|Jersey| NJ|
 4| Jones| 30| Sales| 30| 4|221 Cavalier St|Newark| DE|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+

In [0]:
#Drop duplicate column

empDF.join(addDF,["emp_id"]).show()

+------+--------+-----------+---------------+------+-----+
emp_id| name|emp_dept_id| addline1| city|state|
+------+--------+-----------+---------------+------+-----+
 1| Smith| 10| 1523 Main St| SFO| CA|
 2| Rose| 20| 3453 Orange St| SFO| NY|
 3|Williams| 10| 34 Warner St|Jersey| NJ|
 4| Jones| 30|221 Cavalier St|Newark| DE|
+------+--------+-----------+---------------+------+-----+

In [0]:
deptDF.show()

+---------+-----------+
dept_name|emp_dept_id|
+---------+-----------+
 Finance| 10|
Marketing| 20|
 Sales| 30|
 IT| 40|
+---------+-----------+

In [0]:
#Join Multiple DataFrames
empDF.join(addDF,["emp_id"]) \
     .join(deptDF,empDF["emp_dept_id"] == deptDF["emp_dept_id"]) \
     .show()

+------+--------+-----------+---------------+------+-----+---------+-----------+
emp_id| name|emp_dept_id| addline1| city|state|dept_name|emp_dept_id|
+------+--------+-----------+---------------+------+-----+---------+-----------+
 3|Williams| 10| 34 Warner St|Jersey| NJ| Finance| 10|
 1| Smith| 10| 1523 Main St| SFO| CA| Finance| 10|
 2| Rose| 20| 3453 Orange St| SFO| NY|Marketing| 20|
 4| Jones| 30|221 Cavalier St|Newark| DE| Sales| 30|
+------+--------+-----------+---------------+------+-----+---------+-----------+

In [0]:



#Using Where for Join Condition
empDF.join(deptDF).where(empDF["emp_dept_id"] == deptDF["emp_dept_id"]) \
    .join(addDF).where(empDF["emp_id"] == addDF["emp_id"]) \
    .show()


+------+--------+-----------+---------+-----------+------+---------------+------+-----+
emp_id| name|emp_dept_id|dept_name|emp_dept_id|emp_id| addline1| city|state|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+
 1| Smith| 10| Finance| 10| 1| 1523 Main St| SFO| CA|
 2| Rose| 20|Marketing| 20| 2| 3453 Orange St| SFO| NY|
 3|Williams| 10| Finance| 10| 3| 34 Warner St|Jersey| NJ|
 4| Jones| 30| Sales| 30| 4|221 Cavalier St|Newark| DE|
+------+--------+-----------+---------+-----------+------+---------------+------+-----+

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
         .appName('SparkByExamples.com') \
         .getOrCreate()

In [0]:
spark

Out[12]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:


data = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
columns = ["Name","Dept","Salary"]
df = spark.createDataFrame(data=data,schema=columns)
df.distinct().show()

+-------+---------+------+
 Name| Dept|Salary|
+-------+---------+------+
 James| Sales| 3000|
Michael| Sales| 4600|
 Robert| Sales| 4100|
 Maria| Finance| 3000|
 Scott| Finance| 3300|
 Jen| Finance| 3900|
 Jeff|Marketing| 3000|
 Kumar|Marketing| 2000|
 Saif| Sales| 4100|
+-------+---------+------+

In [0]:
print("Distinct Count: " + str(df.distinct().count()))


Distinct Count: 9

In [0]:
df.select('Name').show() # vs df.select('Name')

+-------+
 Name|
+-------+
 James|
Michael|
 Robert|
 Maria|
 James|
 Scott|
 Jen|
 Jeff|
 Kumar|
 Saif|
+-------+

In [0]:
df.select("Name", "Dept", "Salary").select(countDistinct("Dept","Salary"))

Out[21]: DataFrame[count(DISTINCT Dept, Salary): bigint]

In [0]:
df.select("Name", "Dept", "Salary").select(countDistinct("Dept","Salary")).show()

+----------------------------+
count(DISTINCT Dept, Salary)|
+----------------------------+
 8|
+----------------------------+

In [0]:
df.select("Name", "Dept", "Salary").select(countDistinct("Dept","Salary")).cache().show()

+----------------------------+
count(DISTINCT Dept, Salary)|
+----------------------------+
 8|
+----------------------------+

In [0]:
df.select("Name", "Dept", "Salary").select(countDistinct("Dept","Salary")).show()

+----------------------------+
count(DISTINCT Dept, Salary)|
+----------------------------+
 8|
+----------------------------+

In [0]:

# Using countDistrinct()
from pyspark.sql.functions import countDistinct  
# это очень важная штука
import pyspark.sql.functions as F

df2=df.select(countDistinct("Dept","Salary"))
df2.show()

+----------------------------+
count(DISTINCT Dept, Salary)|
+----------------------------+
 8|
+----------------------------+

In [0]:
df.select(F.count("Dept")).show()

+-----------+
count(Dept)|
+-----------+
 10|
+-----------+

In [0]:
df.select(F.corr("Dept", "Salary")).show()

+------------------+
corr(Dept, Salary)|
+------------------+
 null|
+------------------+

In [0]:




print("Distinct Count of Department &amp; Salary: "+ str(df2.collect()[0][0]))

df.createOrReplaceTempView("PERSON")
spark.sql("select distinct(count(*)) from PERSON").show()

+-------+---------+------+
 Name| Dept|Salary|
+-------+---------+------+
 James| Sales| 3000|
Michael| Sales| 4600|
 Robert| Sales| 4100|
 Maria| Finance| 3000|
 Scott| Finance| 3300|
 Jen| Finance| 3900|
 Jeff|Marketing| 3000|
 Kumar|Marketing| 2000|
 Saif| Sales| 4100|
+-------+---------+------+

Distinct Count: 9
+----------------------------+
count(DISTINCT Dept, Salary)|
+----------------------------+
 8|
+----------------------------+

Distinct Count of Department &amp; Salary: 8
+--------+
count(1)|
+--------+
 10|
+--------+

In [0]:
data = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
columns = ["Name","Dept","Salary"]
df = spark.createDataFrame(data=data,schema=columns)
df.distinct().show()
print("Distinct Count: " + str(df.distinct().count()))

# Using countDistrinct()
from pyspark.sql.functions import countDistinct
df2=df.select(countDistinct("Dept","Salary"))
df2.show()

print("Distinct Count of Department &amp; Salary: "+ str(df2.collect()[0][0]))

df.createOrReplaceTempView("PERSON")
spark.sql("select distinct(count(*)) from PERSON").show()

In [0]:
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
import pyspark.sql.functions as F

In [0]:
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
schema = ["employee_name", "department", "salary"]

In [0]:
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

In [0]:
df

In [0]:
print("avg: " + str(df.select(avg("salary")).collect()[0][0]))

df.select(collect_list("salary")).show(truncate=False)

df.select(collect_set("salary")).show(truncate=False)

df2 = df.select(countDistinct("department", "salary"))
df2.show(truncate=False)
print("Distinct Count of Department &amp; Salary: "+str(df2.collect()[0][0]))



In [0]:
from pyspark.sql.functions import first, last, min, mean
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance, var_samp,  var_pop